In [ ]:
# testing

In [ ]:
import numpy as np
from scipy.integrate import simps

def aod_to_pdf(aod, wavelengths, refractive_index, height, verbose=False):
    """
    Estimates the aerosol probability density function (PDF) from aerosol optical depth (AOD) data using the LEASD algorithm.

    Parameters:
    -----------
    aod: array-like
        The AOD measurement(s) at the specified wavelengths.
    wavelengths: array-like
        The wavelengths at which the AOD is measured in nanometers.
    refractive_index: complex float
        The complex refractive index of the aerosol.
    height: float
        The height of the aerosol layer in kilometers.
    verbose: bool, optional
        Whether to print intermediate results during the inversion process.

    Returns:
    --------
    bin_centers: array-like
        The center values of each bin in the PDF.
    pdf: array-like
        The estimated PDF values for each bin.
    """

    # Convert wavelengths to micrometers
    wavelengths_um = wavelengths / 1000.0

    # Convert AOD to extinction coefficient
    k = 3.34e-4 * aod / np.power(wavelengths_um, 4)

    # Define the bin edges for the PDF
    bin_edges = np.logspace(np.log10(10), np.log10(1000), num=25, base=10)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    # Define the initial guess for the size distribution
    n = np.ones_like(bin_centers)

    # Define the forward model for the extinction coefficient
    def k_model(d, n):
        return 4 * np.pi * np.imag(refractive_index) * d * n

    # Iterate to find the size distribution that best fits the AOD measurement
    for i in range(10):
        # Calculate the extinction coefficient for the current size distribution
        k_calc = k_model(bin_centers, n)

        # Calculate the weight matrix for the inversion
        w = np.diag(1.0 / np.power(k_calc, 2))

        # Calculate the jacobian matrix for the inversion
        j = np.zeros((len(wavelengths), len(n)))
        for j_wl, wl in enumerate(wavelengths_um):
            j[j_wl, :] = 4 * np.pi * np.imag(refractive_index) * wl * n

        # Calculate the misfit between the calculated and observed AOD
        misfit = k_calc - k

        # Calculate the update to the size distribution using the LEASD algorithm
        delta_n = np.linalg.lstsq(w @ j, w @ misfit, rcond=None)[0]

        # Update the size distribution
        n += delta_n

        # Print intermediate results if requested
        if verbose:
            print(f"Iteration {i+1}: RMS error = {np.sqrt(simps(np.power(misfit, 2), wavelengths_um))}")

    # Calculate the PDF from the final size distribution
    pdf = n / np.diff(bin_edges)

    return bin_centers, pdf
